In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
%cd /content/drive/MyDrive/성균관대/Agent/

/content/drive/MyDrive/성균관대/Agent


In [13]:
# 필요한 라이브러리 임포트
import os
import openai
from datetime import datetime
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# OpenAI API Key 설정 (api_key.txt에서 읽어오기)
def load_api_key(file_path="api_key.txt"):
    """
    외부 파일에서 API 키를 읽어옵니다.
    """
    try:
        with open(file_path, "r") as f:
            return f.read().strip()
    except FileNotFoundError:
        print(f"API 키 파일이 {file_path}에 없습니다.")
        return None

API_KEY = load_api_key()
if not API_KEY:
    raise ValueError("API 키를 설정해주세요. 'api_key.txt' 파일을 확인하세요.")

openai.api_key = API_KEY

In [14]:
!pip install --upgrade openai

In [3]:
# Short-Term Memory Class
class ShortTermMemory:
    def __init__(self):
        self.memory = []  # 하루 동안의 관찰 저장

    def add_observation(self, description):
        """
        OpenAI GPT API를 사용하여 Poignancy와 Emotion 정보를 분석한 후 Short-Term Memory에 추가
        """
        poignancy, emotion, emotion_score = self.analyze_with_gpt(description)
        observation = {
            "timestamp": datetime.now(),
            "description": description,
            "poignancy": poignancy,
            "emotion": emotion,
            "emotion_score": emotion_score,
        }
        self.memory.append(observation)
        return observation

    def analyze_with_gpt(self, description):
        """
        OpenAI GPT API를 사용하여 Poignancy와 Emotion 정보를 분석
        """
        prompt = f"""
        Analyze the following observation and determine:
        1. The poignancy of the event (a value between 0.0 and 1.0, where 1.0 is most important).
        2. The primary emotion it conveys (choose from joy, sadness, anger, fear, anticipation, surprise, trust, disgust).
        3. The intensity of the emotion (a value between 0.0 and 1.0).

        Observation: "{description}"

        Return the result in JSON format:
        {{"poignancy": 0.8, "emotion": "joy", "emotion_score": 0.9}}
        """
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                top_p=0.1,
                temperature=0.1,
                messages=[
                    {"role": "system", "content": "You are an assistant for analyzing emotional observations."},
                    {"role": "user", "content": prompt}
                ]
            )
            generated_text = response.choices[0].message.content
            print(f"GPT Generated Text:\n{generated_text}\n")  # 디버깅용 출력

            # JSON 부분만 추출
            start_idx = generated_text.find("{")
            end_idx = generated_text.rfind("}") + 1
            json_text = generated_text[start_idx:end_idx]

            # JSON 변환
            result = eval(json_text)
            return result["poignancy"], result["emotion"], result["emotion_score"]
        except Exception as e:
            print(f"Error parsing GPT output: {e}")
            print("Returning default values.")
            return 0.5, "neutral", 0.5  # 기본값 반환

    def get_memory(self):
        """현재 Short-Term Memory 반환"""
        return self.memory

    def clear_memory(self):
        """하루가 끝난 후 Short-Term Memory 초기화"""
        self.memory = []

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import numpy as np
from datetime import datetime

# Long-Term Memory Class
class LongTermMemory:
    def __init__(self):
        self.memory = []  # 장기 기억 저장소
        self.embedder = SentenceTransformer('all-MiniLM-L6-v2')  # 문장 임베딩 모델 초기화

    def add_memory(self, memory):
        """Reflection에서 전달받은 데이터를 저장"""
        memory["embedding"] = self.embed_text(memory["description"])  # 문장 임베딩 추가
        self.memory.append(memory)

    def embed_text(self, text):
        """문장을 임베딩 벡터로 변환"""
        return self.embedder.encode([text])[0]  # 단일 문장에 대해 임베딩 생성

    def calculate_relevance(self, query_embedding, memory_embedding):
        """문장 임베딩 간 코사인 유사도를 계산하여 관련성 점수 반환"""
        similarity = cosine_similarity([query_embedding], [memory_embedding])[0][0]
        return similarity  # 유사도 값 반환

    def calculate_recency_score(self, memory_time):
        """Recency 점수 계산 (시간이 지날수록 점수가 감소)"""
        time_diff = (datetime.now() - memory_time).total_seconds() / 3600.0  # 시간 차이를 시간 단위로 계산
        return np.exp(-0.005 * time_diff)  # 시간이 오래될수록 점수 감소

    def calculate_emotion_relevance(self, query_emotion, memory_emotion):
        """
        현재 감정(query_emotion)과 메모리 감정(memory_emotion)의 관련성 계산.
        - 같은 감정: 1.0
        - 반대 감정: 0.0
        - 다른 감정: 0.5
        """
        emotion_pairs = {
            "joy": "sadness", "sadness": "joy",
            "anger": "fear", "fear": "anger",
            "anticipation": "surprise", "surprise": "anticipation",
            "trust": "disgust", "disgust": "trust"
        }

        if query_emotion == memory_emotion:  # 같은 감정
            return 1.0
        elif emotion_pairs.get(query_emotion) == memory_emotion:  # 반대 감정
            return 0.0
        else:  # 다른 감정
            return 0.5

    def retrieve_with_llm(self, description, analyze_with_gpt, recency_weight=1.0, poignancy_weight=1.0,
                          emotion_importance_weight=1.0, emotion_relevance_weight=1.0, relevance_weight=1.0):
        """
        사용자 입력(description)을 기반으로 가장 적합한 메모리를 검색
        """
        # 사용자 입력에 대한 임베딩 계산
        query_embedding = self.embed_text(description)

        # LLM으로 감정 분석
        query_poignancy, query_emotion, query_emotion_score = analyze_with_gpt(description)

        # 메모리 점수 계산
        scored_memories = []
        for memory in self.memory:
            recency_score = self.calculate_recency_score(memory["timestamp"])  # 최신성 점수
            poignancy_score = memory["poignancy"]  # 중요도 점수
            emotion_importance = memory["emotion_score"]  # 감정 강도
            emotion_relevance = self.calculate_emotion_relevance(query_emotion, memory["emotion"])  # 감정 관련성
            relevance_score = self.calculate_relevance(query_embedding, memory["embedding"])  # 문장 관련성

            # 최종 점수 계산
            final_score = (
                recency_weight * recency_score +
                poignancy_weight * poignancy_score +
                emotion_importance_weight * emotion_importance +
                emotion_relevance_weight * emotion_relevance +
                relevance_weight * relevance_score
            )
            scored_memories.append((final_score, memory))  # 점수와 메모리 저장

        # 점수에 따라 내림차순 정렬
        scored_memories.sort(key=lambda x: x[0], reverse=True)
        return [memory for _, memory in scored_memories]  # 정렬된 메모리 반환

    def get_memory(self):
        """현재 Long-Term Memory 반환"""
        return self.memory

    def clear_memory(self):
        """장기 메모리 초기화"""
        self.memory = []


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [7]:
# Reflection Function
def reflect(short_term_memory, emotion_threshold=0.5, poignancy_threshold=0.5):
    """
    Short-Term Memory에서 Emotion Score와 Poignancy를 기준으로 Long-Term Memory로 저장할 데이터를 선택
    """
    reflected_memories = []
    for memory in short_term_memory:
        if memory["emotion_score"] >= emotion_threshold and memory["poignancy"] >= poignancy_threshold:
            reflected_memories.append(memory)
    return reflected_memories

In [9]:
# 메인 실행
if __name__ == "__main__":
    # Short-Term Memory 생성
    stm = ShortTermMemory()
    print("Short-Term Memory System Initialized.\n")

    # 사용자 입력을 통한 Observation 추가
    while True:
        description = input("관찰 내용을 입력하세요 (종료하려면 'end' 입력): ")
        if description.lower() == "end":
            print("관찰 입력을 종료합니다...\n")
            break

        observation = stm.add_observation(description)
        print(f"Short-Term Memory에 추가됨: {observation}\n")

    # Reflection 수행
    print("\n--- Reflection 시작 ---")
    ltm = LongTermMemory()
    reflected_memories = reflect(stm.get_memory(), emotion_threshold=0.5, poignancy_threshold=0.5)
    for memory in reflected_memories:
        ltm.add_memory(memory)
    print(f"Reflection 완료: Long-Term Memory에 저장된 메모리: {reflected_memories}\n")

    # LLM 기반 Retrieval
    print("--- Retrieval 시작 ---")
    retrieval_description = input("현재 상황이나 감정을 설명해주세요: ")
    retrieved_memories = ltm.retrieve_with_llm(retrieval_description, stm.analyze_with_gpt)
    print(f"Retrieved Memories: {retrieved_memories}\n")

    # Short-Term Memory 초기화
    stm.clear_memory()
    print("Short-Term Memory가 초기화되었습니다.")

Short-Term Memory System Initialized.

관찰 내용을 입력하세요 (종료하려면 'end' 입력): I couldn't sleep last night because I was overthinking about work.
GPT Generated Text:
```json
{"poignancy": 0.7, "emotion": "sadness", "emotion_score": 0.8}
```

Short-Term Memory에 추가됨: {'timestamp': datetime.datetime(2024, 12, 3, 6, 51, 17, 163893), 'description': "I couldn't sleep last night because I was overthinking about work.", 'poignancy': 0.7, 'emotion': 'sadness', 'emotion_score': 0.8}

관찰 내용을 입력하세요 (종료하려면 'end' 입력): I had coffee in the morning to stay awake, but now I feel jittery.
GPT Generated Text:
```json
{"poignancy": 0.4, "emotion": "fear", "emotion_score": 0.6}
```

Short-Term Memory에 추가됨: {'timestamp': datetime.datetime(2024, 12, 3, 6, 52, 47, 328016), 'description': 'I had coffee in the morning to stay awake, but now I feel jittery.', 'poignancy': 0.4, 'emotion': 'fear', 'emotion_score': 0.6}

관찰 내용을 입력하세요 (종료하려면 'end' 입력): I tried meditating during lunch, but it was hard to focus.
GPT Generated T

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Reflection 완료: Long-Term Memory에 저장된 메모리: [{'timestamp': datetime.datetime(2024, 12, 3, 6, 51, 17, 163893), 'description': "I couldn't sleep last night because I was overthinking about work.", 'poignancy': 0.7, 'emotion': 'sadness', 'emotion_score': 0.8, 'embedding': array([-2.46210825e-02,  3.09327468e-02,  1.61800312e-03,  7.38129243e-02,
        1.33476883e-01, -1.75710395e-02,  2.70095980e-03, -1.89785063e-02,
       -1.03102007e-03,  2.83794338e-03, -1.13071308e-01,  7.83485845e-02,
       -1.22190863e-02,  3.50625888e-02, -3.61168832e-02, -6.47739917e-02,
        2.52011605e-02, -7.92112947e-03, -1.86186247e-02,  1.62897464e-02,
        1.08058490e-02,  1.91760641e-02, -2.33682804e-03,  6.19586781e-02,
        4.14774753e-02,  1.10476837e-01, -5.34906089e-02, -2.72697974e-02,
       -2.89944056e-02, -6.84139431e-02, -6.54531121e-02,  8.87829345e-04,
       -5.61125949e-02, -3.51941437e-02,  7.48493820e-02, -6.41231984e-02,
       -3.66110122e-03,  4.68078107e-02,  6.36744723e-02,